 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Naszym zadaniem tutaj jest pobranie udostępnionych nam danych do obszaru roboczego, które w następnym kroku wgramy na naszą bazę danych. Ich obróbka oraz analiza zostanie przeprowadzona w specjalnie do tego celu przygotowanych kolejnych notatnikach.

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/v2. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/v2/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: `iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk`.

 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 1000/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

 Tutaj zaimportuj wymagane biblioteki

In [2]:
import time
import requests
import pandas as pd
import json
import csv
import os
import signal

 Tutaj zdefiniuj paramatry połączenia do API

In [3]:
url_airport = 'https://api-datalab.coderslab.com/api/v2/airport/'
url_weather ='https://api-datalab.coderslab.com/api/v2/airportWeather?date='
url_aircraft = 'https://api-datalab.coderslab.com/api/v2/aircraft'
url_flight = 'https://api-datalab.coderslab.com/api/v2/flight?airportId='
headers = {'Authorization':'iKRsQ8vdqgT903o2vH1rsejOeQ0F7YC9TvutH6Wk','accept': 'application/json'}

 Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [4]:
os.chdir('../data')
df_airports_ids = pd.read_csv("airports.csv",sep =' ')
os.chdir('../notebooks')
os.getcwd()
df_airports_ids

,origin_airport_id
0,10874
1,11233
2,13360
3,15008
4,11638
...,...
359,13303
360,13795
361,10693
362,12945


In [6]:
# testowo zrzucam sobie wartości do array 
origin_airport_list = df_airports_ids['origin_airport_id'].values


In [7]:
# testowo zrzucam array do series potem do series
series = pd.Series(df_airports_ids['origin_airport_id'].values)
type(series)

pandas.core.series.Series

 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

In [5]:
airports_list = []
for airport_id in df_airports_ids['origin_airport_id'].values:
    try:
        response = requests.get(f'{url_airport}{airport_id}',headers=headers)
        if response.status_code == 200:
            print('Połączenie udane. Dołączam lotnisko do listy')
            json_airports_ids = response.json()
            airports_list.append(json_airports_ids)
        else:
            print('Wystąpił błąd. Kod odpowiedzi:', response.status_code)
    except: 
        if response.status_code == 200:
            print('Połączenie udane. Kod odpowiedzi:', response.status_code)
        else:
            print('Wystąpił błąd. Kod odpowiedzi:', response.status_code)


Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi:

Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Wystąpił błąd. Kod odpowiedzi: 400
Połączenie udane. Dołączam lotnisko do listy
Wystąpił b

In [5]:
len(airports_list)

97

### *** ZRZUT DANYCH DO DF, zwróc uwage że id airportu jest zachowane jako int co widac potem w BD i nie wiem czy nei trzeba tego zamienic na str***


In [6]:
airport_df = pd.DataFrame.from_records(airports_list, index = None,\
                                      exclude = None, columns = None, coerce_float = False, nrows = None)
airport_df.drop_duplicates()

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
0,11638,Fresno Air Terminal,"Fresno, CA","FRESNO YOSEMITE INTERNATIONAL, CA US"
1,13342,General Mitchell Field,"Milwaukee, WI","MILWAUKEE MITCHELL AIRPORT, WI US"
2,13244,Memphis International,"Memphis, TN","MEMPHIS INTERNATIONAL AIRPORT, TN US"
3,15096,Syracuse Hancock International,"Syracuse, NY","SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US"
4,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
...,...,...,...,...
92,13198,Kansas City International,"Kansas City, MO","KANSAS CITY INTERNATIONAL AIRPORT, MO US"
93,10423,Austin - Bergstrom International,"Austin, TX","AUSTIN BERGSTROM INTERNATIONAL AIRPORT, TX US"
94,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
95,13303,Miami International,"Miami, FL","MIAMI INTERNATIONAL AIRPORT, FL US"


***sprawdzam typy danych***

In [7]:
airport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ORIGIN_AIRPORT_ID     97 non-null     int64 
 1   DISPLAY_AIRPORT_NAME  97 non-null     object
 2   ORIGIN_CITY_NAME      97 non-null     object
 3   NAME                  97 non-null     object
dtypes: int64(1), object(3)
memory usage: 3.2+ KB


***powyżej widzę że nie ma pustych wierszy,  jedyne co zamieniam to typ danych kolumny 'ORIGIN_AIRPORT_ID' na string***

In [8]:
airport_df['ORIGIN_AIRPORT_ID']= airport_df['ORIGIN_AIRPORT_ID'].astype('str')

# poniższa ramka jest zbędna to tylko test na zapis listy do pliku

In [9]:
# with open('airport.csv', 'a',newline='') as file:
#     coursor = csv.DictWriter(file, fieldnames =airports_list[0].keys())
#     coursor.writeheader()
#     coursor.writerows(airports_list)

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić czy ta część została poprawnie wykonana

In [10]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`

### *** ZRZUT DF DO PLIKU,  KONIECZNIE OPRACOWAC JAK ZRZUĆ PLIK DO ŚCIEŻKI WZGLĘDNEJ ***

In [11]:
os.chdir('../data/raw')
airport_df.to_csv('airport_list.csv', index = False)

 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

In [7]:
months_list =['2019-01','2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08'\
             ,'2019-09','2019-10','2019-11','2019-12','2020-01','2020-02','2020-03']
# months_list

In [13]:
weather_API_list = []

for month in months_list:
    try:
        response = requests.get(f'{url_weather}{month}'\
                       ,headers=headers)
        if response.status_code == 200:
            print('Połączenie udane. Dołączam dane pogodowe do listy')
            json_weather = response.json()
#             PONIŻEJ MUSIAŁEM UZYC += BO JAK UŻYLEM APPEND TO ROBIŁA MI SIE LISTA[LISTA{SŁOWNIK}]
            weather_API_list += json_weather
            time.sleep(0.01)
        else:
            print('Wystąpił błąd. Kod odpowiedzi:', response.status_code)
    except: 
        if response.status_code == 200:
            print('Połączenie udane. Kod odpowiedzi:', response.status_code)
        else:
            print('Wystąpił błąd. Kod odpowiedzi:', response.status_code)

Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy
Połączenie udane. Dołączam dane pogodowe do listy


In [18]:
# weather_API_list = weather_API_list[0]
airport_weather_df = pd.DataFrame.from_records(weather_API_list)
airport_weather_df.drop_duplicates().head()

,WT18,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,...,PGTM,WT10,WESD,SN32,SX32,PSUN,TSUN,TOBS,WT07,WT11
0,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-01,4.70,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-02,4.92,0.57,0.0,0.0,56.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-03,5.37,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-04,12.08,1.44,0.0,0.0,56.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-05,13.42,0.00,0.0,0.0,49.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
airport_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46226 entries, 0 to 46225
Data columns (total 33 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   WT18     0 non-null      object 
 1   STATION  46226 non-null  object 
 2   NAME     46226 non-null  object 
 3   DATE     46226 non-null  object 
 4   AWND     45845 non-null  float64
 5   PRCP     46197 non-null  float64
 6   SNOW     32338 non-null  float64
 7   SNWD     31750 non-null  float64
 8   TAVG     34625 non-null  float64
 9   TMAX     46203 non-null  float64
 10  TMIN     46200 non-null  float64
 11  WDF2     45854 non-null  float64
 12  WDF5     45704 non-null  float64
 13  WSF2     45854 non-null  float64
 14  WSF5     45704 non-null  float64
 15  WT01     16798 non-null  float64
 16  WT08     5589 non-null   float64
 17  WT02     2268 non-null   float64
 18  WT03     5085 non-null   float64
 19  WT04     362 non-null    float64
 20  WT09     316 non-null    float64
 21  WT06     522

***typy danych wygladaja OK, niektóre z kolumn mają sporo nulli, zostawiam je***

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [20]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`

In [23]:
os.chdir('../data/raw')
weather_df.to_csv('airport_weather.csv', index = False)
os.chdir('../../notebooks')

 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft`, a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


In [24]:
aircrafts_list =[]

try:
    response =requests.get(url_aircraft,headers=headers)
    if response.status_code == 200:
        print('Połączenie udane. Dołączam dane aircraft do listy')
        json_aircraft = response.json()
        aircrafts_list += json_aircraft
#             time.sleep(1)
    else:
        print('Wystąpił błąd. Kod odpowiedzi:', response.status_code)
except: 
    if response.status_code == 200:
        print('Połączenie udane. Kod odpowiedzi:', response.status_code)
    else:
        print('Wystąpił błąd. Kod odpowiedzi:', response.status_code)

Połączenie udane. Dołączam dane aircraft do listy


In [25]:
aircraft_df = pd.DataFrame.from_records(aircrafts_list)
aircraft_df

,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0
...,...,...,...
7378,2019,N14011,337.0
7379,2019,N16008,337.0
7380,2019,N16009,337.0
7381,2019,N2250U,276.0


In [26]:
aircraft_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7383 entries, 0 to 7382
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MANUFACTURE_YEAR  7383 non-null   int64  
 1   TAIL_NUM          7383 non-null   object 
 2   NUMBER_OF_SEATS   7376 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 173.2+ KB


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [27]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`

In [29]:
os.chdir('../data/raw')
aircraft_df.to_csv('aircraft.csv', index = False)
os.chdir('../../notebooks')
os.getcwd()

'C:\\Users\\aroma\\google\\Colab Notebooks\\Pliki_do_projektu_końcowego\\notebooks'

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku,
 - W przypadku, gdy nie ma dostępnych danych dla danego lotniska, API zwraca kod [204](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/204), w ten sposób możesz pominąć lotniska, dla których dane nie są dostępne,
 - Pobranie całości danych zajmuje dłuższą chwilę, zanim włączysz pętle dla wszystkich danych, sprawdź pobieranie danych dla jednego, dwóch lotnisk aby uniknąć frustracji.

#    tworze listę tylko tych airport_id, które występują w API żeby nie odpytywac o wszystkie

In [1]:
airports_list2 = []
for airport in airports_list:
    airports_list2.append(airport['ORIGIN_AIRPORT_ID'])
    

NameError: name 'airports_list' is not defined

# pętlimy przez airporty i miesiące 

In [31]:
def handle_interrupt(signal, frame):
    raise KeyboardInterrupt
flights_list = []
for airport in airports_list2:
    for month in months_list:
        try:
            response = requests.get(f'{url_flight}{airport}&date={month}',headers = headers)
            response.raise_for_status()
            time.sleep(0.01)
            if response.status_code == 200:
                print(f'Połączenie udane. Dołączam flight w miesiącu {month} z airport {airport} do listy ')
                json_flight = response.json()
                flights_list += json_flight
            else:
                print('Wystąpił błąd. Kod odpowiedzi:', response.status_code)

        except KeyboardInterrupt:
            print("Przerwano zapytanie na żądanie użytkownika.")
        except:
            if response.status_code == 200:
                print('Połączenie udane. Kod odpowiedzi:', response.status_code)
            else:
                print(f'Wystąpił błąd. w miesiącu {month} z airport {airport} Kod odpowiedzi:', response.status_code)          


Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod o

Połączenie udane. Dołączam flight w miesiącu 2020-03 z airport 11292 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-01 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-02 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-03 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-04 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-05 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-06 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-07 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-08 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-09 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-10 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-11 z airport 11259 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-12

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod o

Połączenie udane. Dołączam flight w miesiącu 2020-02 z airport 12889 do listy 
Połączenie udane. Dołączam flight w miesiącu 2020-03 z airport 12889 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-01 z airport 14492 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-02 z airport 14492 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-03 z airport 14492 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-04 z airport 14492 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-05 z airport 14492 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-06 z airport 14492 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-07 z airport 14492 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-08 z airport 14492 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-09 z airport 14492 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-10 z airport 14492 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-11

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Połączenie udane. Dołączam flight w miesiącu 2019-01 z airport 11278 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-02 z airport 11278 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-03 z airport 11278 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-04 z airport 11278 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-05 z airport 11278 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-06 z airport 11278 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-07 z airport 11278 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-08 z airport 11278 do listy 
Połączenie udane. 

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Połączenie udane. Dołączam flight w miesiącu 2019-01 z airport 12758 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-02 z airport 12758 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-03 z airport 12758 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-04 z airport 12758 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-05 z airport 12758 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-06 z airport 12758 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-07 z airport 12758 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-08 z airport 12758 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-09 z airport 12758 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-10 z airport 12758 do listy 
Połączenie udane. Dołączam flight w

Połączenie udane. Dołączam flight w miesiącu 2019-10 z airport 12173 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-11 z airport 12173 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-12 z airport 12173 do listy 
Połączenie udane. Dołączam flight w miesiącu 2020-01 z airport 12173 do listy 
Połączenie udane. Dołączam flight w miesiącu 2020-02 z airport 12173 do listy 
Połączenie udane. Dołączam flight w miesiącu 2020-03 z airport 12173 do listy 
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
P

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Połączenie udane. Dołączam flight w miesiącu 2019-01 z 

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Połączenie udane. Dołączam flight w miesiącu 2019-01 z airport 12478 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-02 z airport 12478 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-03 z airport 12478 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-04 z airport 12478 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-05 z airport 12478 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-06 z airport 12478 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-07 z airport 12478 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-08 z airport 12478 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-09 z airport 12478 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-10 z airport 12478 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-11 z airport 12478 do listy 
Połączenie udane. Dołączam

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Połączenie udane. Dołączam flight w miesiącu 2019-01 z airport 13303 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-02 z airport 13303 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-03 z airport 13303 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-04 z airport 13303 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-05 z airport 13303 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-06 z airport 13303 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-07 z airport 13303 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-08 z airport 13303 do listy 
Połączenie udane. Dołączam flight w miesiącu 2019-09 

alternatywne pobranie testowe wszystkich airport,  nie tylko tych 

In [8]:
flights_list_2 = []
for airport in df_airports_ids['origin_airport_id'].values:
    for month in months_list:
        try:
            response = requests.get(f'{url_flight}{airport}&date={month}',headers = headers)
            response.raise_for_status()
            time.sleep(0.01)
            if response.status_code == 200:
                print(f'Połączenie udane. Dołączam flight w miesiącu {month} z airport {airport} do listy ')
                json_flight = response.json()
                flights_list_2 += json_flight
            else:
                print('Wystąpił błąd. Kod odpowiedzi:', response.status_code)

        except KeyboardInterrupt:
            print("Przerwano zapytanie na żądanie użytkownika.")
        except:
            if response.status_code == 200:
                print('Połączenie udane. Kod odpowiedzi:', response.status_code)
            else:
                print(f'Wystąpił błąd. w miesiącu {month} z airport {airport} Kod odpowiedzi:', response.status_code)          


Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod o

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod o

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod o

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod o

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod o

...

Wystąpił błąd. w miesiącu 2020-02 z airport 13930 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2020-03 z airport 13930 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-01 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-02 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-03 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-04 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-05 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-06 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-07 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-08 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-09 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-10 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-11 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-12 z airport 11997 Kod odpowiedzi: 204
Wystąpił błąd. w mie

Wystąpił błąd. w miesiącu 2019-04 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-05 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-06 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-07 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-08 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-09 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-10 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-11 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-12 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2020-01 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2020-02 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2020-03 z airport 10685 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-01 z airport 14842 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-02 z airport 14842 Kod odpowiedzi: 204
Wystąpił błąd. w mie

Wystąpił błąd. w miesiącu 2019-07 z airport 10409 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-08 z airport 10409 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-09 z airport 10409 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-10 z airport 10409 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-11 z airport 10409 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-12 z airport 10409 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2020-01 z airport 10409 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2020-02 z airport 10409 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2020-03 z airport 10409 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-01 z airport 12519 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-02 z airport 12519 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-03 z airport 12519 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-04 z airport 12519 Kod odpowiedzi: 204
Wystąpił błąd. w miesiącu 2019-05 z airport 12519 Kod odpowiedzi: 204
Wystąpił błąd. w mie

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod o

Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204
Wystąpił błąd. Kod odpowiedzi: 204


In [10]:
flight_df_2 = pd.DataFrame.from_records(flights_list_2)

In [33]:
flight_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1379543 entries, 0 to 1379542
Data columns (total 27 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   MONTH                1379543 non-null  int64  
 1   DAY_OF_MONTH         1379543 non-null  int64  
 2   DAY_OF_WEEK          1379543 non-null  int64  
 3   OP_UNIQUE_CARRIER    1379543 non-null  object 
 4   TAIL_NUM             1368552 non-null  object 
 5   OP_CARRIER_FL_NUM    1379543 non-null  int64  
 6   ORIGIN_AIRPORT_ID    1379543 non-null  int64  
 7   DEST_AIRPORT_ID      1379543 non-null  int64  
 8   CRS_DEP_TIME         1379543 non-null  int64  
 9   DEP_TIME             1341181 non-null  float64
 10  DEP_DELAY_NEW        1341180 non-null  float64
 11  DEP_TIME_BLK         1379543 non-null  object 
 12  CRS_ARR_TIME         1379543 non-null  int64  
 13  ARR_TIME             1340200 non-null  float64
 14  ARR_DELAY_NEW        1338111 non-null  float64
 15

***pomimo szerszego odczytania API i tak mam za mało wierszy  1379543 zamiast 1386120***

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [34]:
flight_df_expected_shape = (1386120, 27)
assert flight_df_expected_shape == flight_df.shape

AssertionError: 

***NIE WIEM SKĄD RÓZNICA W ILOŚCI 657700 WIERSZY***
O TYLE MNIEJSZĘ BĘDĄ DATAFRAMY

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`

In [ ]:
os.chdir('../data/raw')
flight_df.to_csv('flight.csv', index = False)
os.chdir('../../notebooks')
os.getcwd()

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [35]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)

Wszystko wygląda OK :) Możesz przejść do kolejnego kroku.
